# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [10]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,santa terezinha,-29.9002,-50.0904,292.60,93,100,9.65,BR,1701205593
1,1,ushuaia,-54.8000,-68.3000,279.96,81,75,12.35,AR,1701206187
2,2,hermanus,-34.4187,19.2345,290.81,69,47,5.78,ZA,1701206173
3,3,port-aux-francais,-49.3500,70.2167,278.87,98,100,13.59,TF,1701206181
4,4,kourou,5.1552,-52.6478,302.18,80,21,3.46,GF,1701206188


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [28]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 2,
)

# Display the map
city_map

ImportError: Geographic projection support requires: cartopy, geoviews, pyproj.

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [17]:
# Narrow down cities that fit criteria and drop any results with null values
ideal = city_data_df.loc[city_data_df['Max Temp'] < 290]

# Drop any rows with null values

ideal.dropna(inplace=True)
# Display sample data
ideal

/var/folders/5x/5lkn2ztj7v54fkzpt5p45b740000gn/T/ipykernel_81365/111890418.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal.dropna(inplace=True)


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,ushuaia,-54.8000,-68.3000,279.96,81,75,12.35,AR,1701206187
3,3,port-aux-francais,-49.3500,70.2167,278.87,98,100,13.59,TF,1701206181
5,5,college,64.8569,-147.8028,265.62,89,75,1.54,US,1701206188
7,7,talnakh,69.4865,88.3972,249.49,97,31,2.05,RU,1701206179
10,10,grytviken,-54.2811,-36.5092,273.26,67,7,4.44,GS,1701206146
...,...,...,...,...,...,...,...,...,...,...
579,579,tanumah,27.1000,44.1333,289.41,55,63,4.16,SA,1701206463
580,580,vorkuta,67.5000,64.0000,260.46,91,100,3.00,RU,1701206464
581,581,dunedin,-45.8742,170.5036,284.15,57,26,4.02,NZ,1701206173
584,584,weiser,44.2510,-116.9693,276.29,63,6,0.45,US,1701206466


### Step 3: Create a new DataFrame called `hotel_df`.

In [27]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
0,santa terezinha,BR,-29.9002,-50.0904,93,
1,ushuaia,AR,-54.8000,-68.3000,81,
2,hermanus,ZA,-34.4187,19.2345,69,
3,port-aux-francais,TF,-49.3500,70.2167,98,
4,kourou,GF,5.1552,-52.6478,80,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [29]:
# Set parameters to search for a hotel
radius = 10000

params = {
     "categories": 'accommodation.hotel',
    "limit": 1,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df['Lng']
    latitude = hotel_df['Lat']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f'circle: {longitude}, {latitude}, {radius}'
    params["bias"] = f'proximity: {longitude},{latitude}'
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params).json()
    
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
santa terezinha - nearest hotel: No hotel found
ushuaia - nearest hotel: No hotel found
hermanus - nearest hotel: No hotel found
port-aux-francais - nearest hotel: No hotel found
kourou - nearest hotel: No hotel found
college - nearest hotel: No hotel found
flying fish cove - nearest hotel: No hotel found
talnakh - nearest hotel: No hotel found
georgetown - nearest hotel: No hotel found
adamstown - nearest hotel: No hotel found
grytviken - nearest hotel: No hotel found
arkhangel'sk - nearest hotel: No hotel found
dzerzhinsk - nearest hotel: No hotel found
reggane - nearest hotel: No hotel found
qaqortoq - nearest hotel: No hotel found
papatowai - nearest hotel: No hotel found
colonia - nearest hotel: No hotel found
kalianget - nearest hotel: No hotel found
hawaiian paradise park - nearest hotel: No hotel found
ust'-kut - nearest hotel: No hotel found
waitangi - nearest hotel: No hotel found
anadyr - nearest hotel: No hotel found
edinburgh of the seven seas - neare

,City,Country,Lat,Lng,Humidity,Hotel Name
0,santa terezinha,BR,-29.9002,-50.0904,93,No hotel found
1,ushuaia,AR,-54.8000,-68.3000,81,No hotel found
2,hermanus,ZA,-34.4187,19.2345,69,No hotel found
3,port-aux-francais,TF,-49.3500,70.2167,98,No hotel found
4,kourou,GF,5.1552,-52.6478,80,No hotel found
...,...,...,...,...,...,...
581,dunedin,NZ,-45.8742,170.5036,57,No hotel found
582,maceio,BR,-9.6658,-35.7353,73,No hotel found
583,kudahuvadhoo,MV,2.6708,72.8944,79,No hotel found
584,weiser,US,44.2510,-116.9693,63,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [31]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "longitude",
    "latitude",
    geo = True,
    tiles = "OSM")

# Display the map
map_plot

ImportError: Geographic projection support requires: cartopy, geoviews, pyproj.